## Questions de bases sur `numpy` 


**Exercice 1 :** (*Quelques rappels*)  
* Générer un tableau de 100 `0`.
* Remplacer la valeur 42 par `1`.
* Remplacer les 10 premières valeurs par `27`.
* Remplacer les 10 dernières valeurs par `-4`.
* Remplacer une valeur sur 2 par le double.
* Utilisé la méthode `reshape` pour obtenir un tableau de taille `5x20`.
* Multiplier une colonne sur 3 par `-1`.
* Retrancher `-3` aux valeurs non-nulles.
* Calculer la somme des valeurs dans le tableau.

### Opérations sur les tableaux

#### Opérations terme-à-terme

**Exercice 2 :** (*Courbe paramétrique*)  
Tracer la courbe $x = cos(t)$ et $y=sin(t)$ pour $t \in [0, 2\pi]$.  
On pourra utiliser la constante $\pi=$`np.pi`.

#### Broadcasting
Que donnerait les deux cas suivant?

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 

A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 1 x 5
Result (4d array):  
```

**Exercice 3 :** (*Opération terme-à-terme*)

Sans utiliser de boucles (`for/while`) :

 * Créer une matrice (5x6) aléatoire
 * Remplacer une colonne sur deux par sa valeur moins le double de la colonne suivante
 * Remplacer les valeurs négatives par 0 en utilisant un masque binaire


**Exercice 4 :** (*Résolution d'un système linéaire*)  
Résoudre le systeme d'équation: $\begin{cases}3x -2y +z &= 10\\x +5y + 10z &= 21\\y - z &= -5\\\end{cases}$.  
(*Astuce:* utiliser la fonction `np.linalg.inv`).

**Exercice 5 :** (*Blanchiment de donnée*)  
Créer un tableau X de taille `100x5` selon une loi normale ($\sim$ `np.random.randn`).  
Soustrayer à chaque colone sa moyenne et la diviser par son écart type.

# 2 - Plus proche centroid

Nous allons maintenant implémenter l'agorithme des [plus proches centroids](https://en.wikipedia.org/wiki/Nearest_centroid_classifier) en pure `numpy`, depuis la génération de donnée jusqu'a la visualisation du résultat.  
L'idée de cette algorithme est simple:
* A partir de l'ensemble d'entrainement $(X^i, y^i)$, calculer pour chaque classe la moyenne des points de cette classe, *i.e.* $\bar X_l = \frac{1}{|C_l|} \sum_{i\in C_l} X^i$ où $C_l$ est les indices des points appartenant à la classe $l$.
* Pour un point de test $X$, lui assigner la classe $y = \arg\min_l \|X - \bar X_l\|_2$.

Les étapes à réaliser sont:

1) Générer des nuages de points en 2D selon des lois normales de moyennes différentes pour chaque classe.

2) Visualiser ces nuages de points

3) Diviser les données entre un ensemble d'entrainement et un ensemble de test.

4) Entrainer le modèle: calculer les centroids pour chacune des classes

5) Utiliser le modèle: prédire les classes des points dans l'ensemble de test.

6) Évaluer le modèle: donner la précision et le rappel du modèle.


### 2.1 - Génération des données

On va maintenant utiliser les fonctions que l'on a vu précédement pour générer des données pour notre algorithme de plus proche centroid.  
* On va créer des données avec 2 classes : $\{0, 1\}$.
* Pour la classe $i$, on générera les X associés dans $\mathbb R^2$ selon la loi normale $\mathcal N(\mu_i, \pmb I_2)$.


Procédez selon les étapes suivantes:
* Créer 2 vecteurs avec en dimension 2 correspondant aux moyennes de chacune des classes. On commencera avec de moyenne deterministe $\mu_0 = 0$ et $\mu_1 = [rho, rho]$ avec $rho=3$.
* Concaténer `mu0` et `mu1` en un vecteur `mu` de taille `n_classes` x `n_dims` (Indice: utiliser `np.concatenate`).
* Générer un vecteur de classes `y` de taille `n_points=1000` avec des valeurs uniforme dans $\{0, 1\}$.
* Générer un jeu de donné $X$ avec `n_points=1000` points en dimension `n_dim=2` selon la loi $\mathcal N(0, \pmb I_2)$. (*Rappel:* tiré selon une loi normal de covariance $\pmb I_2$ revient à tirer chaque coordonnée selon une loi normale indépendante.)
* Ajouter à chaque point la moyenne correspondant à sa classe.

In [ ]:
# Définion des constantes
rho = 3
n_dim = 2
n_points = 1000

In [ ]:
# A vous de jouer!

### 2.2 - Visualisation des données

Utiliser la fonction `plot_data` pour visualiser les données.  
* Afficher les donnés $(X, y)$, avec l'option `alpha=.1`.
* Afficher les moyennes avec les mêmes couleurs et l'option `s=256, alpha=1`.
Que fait la fonction scatter?

Bonus:

* Visualiser la fonction de répartition des données à l'aide de `plt.contourf`.

In [ ]:
MARKERS = ['^', 's', 'o', 'h', '>', 'v', '*', '+', 'x', '<']
CMAP = plt.get_cmap('tab10', 10)

def plot_data(X, y, alpha=1, s=36):
    """Plot the data with color depending on the classes.
    
    Parameter
    =========
    X: ndarray, shape (n_points, n_dim)
        Point to display
    y : ndarray, shape (n_points,)
        Classes for each point
    alpha: float
        Opacity of the points
    s: int
        Size of the points in pt^2.
    """
    for i in np.unique(y):
        plt.scatter(X[y == i, 0], X[y == i, 1], s=s,
                    c=CMAP(i % 10), alpha=alpha,
                    marker=MARKERS[i % 10])


### 2.3 - Séparation en ensemble d'entrainement et de test

Pour les techniques d'apprentissage supervisé, il est necessaire d'utiliser un ensemble pour apprendre le modèle `X_train, y_train` et un ensemble pour l'évaluer `X_test, y_test`. Créer ces 2 ensembles tirant aléatoirement 60% des donnés pour le train et le reste pour le test. On pourra utiliser soit:
* Un mask booleen `mask_train` tiré aléatoirement à partir d'une loi uniforme comparée à `0.5`.
* La fonction `np.choice` pour créer un ensemble `id_train`


### 2.4 - Entrainement de la plus proche moyenne

Calculer les moyennes par classes:
* On créera d'abord un tableau vide de taille `n_classes` x `n_dim`.
* Calculer les moyennes empirique $\bar \mu_i = \sum_{j \in C_i} X_j$ de `X_train` pour chaque classe $C_i$ .
* Afficher les resultats avec la fonction `plot_data`.

 ### 2.5 - Prédictions

Générer les prédicitons à partir de ce modèle pour l'ensemble `X_test`:
* Pour chaque classe, calculer la distance de la moyenne empirique $\bar \mu_i$ à tous les points et la stockée dans une matrice `C` de taille `n_points` x `n_classes`.
* Pour chaque point, prédite $\bar y = \arg\min_i \| X - \bar \mu_i\|_2^2$
* Afficher les resultats avec la fonction `plot_data`.

### 2.6 - Évaluation du model

Maintenant que l'on a fait des prédictions, on va calculer l'accuracy du modèle. Évaluer les prédictions du modèle entrainé précédement avec la formule: $P(\bar y) = \frac{1}{N}\sum_{i} 1\{\bar y_i = y_i\}$.

### 2.7 - Fonctions

On va maintenant regrouper tout ce qu'on a fait auparavent dans des fonctions.

1) Faire une fonction `generate_data` qui prend en entrée `n_points` et `mu` et qui retourne un jeu de donnée `X, y` comme généré dans 2.1. On rappelle que:
* `mu` est de taille `n_classes` x `n_dim`.
* Pour chaque point, `y` est tiré aléatoirement entre `n_classes`.
* `X` est généré selon une loi normale de moyenne `mu[y]` et de variance $\pmb I$.


2) Écrire une fonction `split_train_test` qui prenne en entrée `(X, y)` et retourne 2 ensemble de données `(X_train, y_train)` et `(X_test, y_test)`. On pourra ajouter un paramètre `ratio` (float in [0, 1]) qui donne le rapport de proportion train/test.

3) Écrire une fonction `fit` qui prenne en entrée `X_train, y_train` et qui renvoie les centroids de chacune des classes `centroids`.

4) Écrire une fonction `predict` qui prenne en entrée `X_test` et `centroids` et qui renvoie en sortie la prédiction `y_pred` du modèle.

5) Faire une fonction `score` qui prend en entrée un couple `y` et `y_pred` et qui retourne l'accuracy pour ce couple.

### 2.8 - Evaluation du model

* Faire jouer les paramètres `n_classes` et `sig`. Quel sont les situations les plus complexes pour le modèle?

* Ajouter une ligne dans le second plot avec la performance de la chance.

* Est ce que le modèle apprends toujours mieux que la chance? Même pour `sig` petit?

In [ ]:
# Parameters
n_classes = 5
n_dims = 2
n_points = 1000
sig = 3


# Generate class mean
mu = sig * np.random.randn(n_classes, n_dims)

list_ratio = np.logspace(-1, 1, 11)

X_test, y_test = generate_data(n_points, mu)
assert len(np.unique(y_test)) == n_classes

plot_data(X_test, y_test, alpha=.1)
plot_data(mu, range(n_classes), s=256)

plt.figure()
score_train = []
score_test = []
for ratio in list_ratio:
    n_train = int(n_points * ratio)
    X_train, y_train = generate_data(n_train, mu)
    centroids = fit(X_train, y_train)
    assert centroids.shape == (n_classes, n_dims)
    y_hat_train = predict(X_train, centroids)
    y_hat_test = predict(X_test, centroids)
    score_train.append(score(y_hat_train, y_train))
    score_test.append(score(y_hat_test, y_test))
    
plt.plot(list_ratio, score_train, label='Train')
plt.plot(list_ratio, score_test, label='Test')
plt.hlines(score(predict(X_test, mu), y_test),
           0, 10, color='k', linestyle='--')
plt.xlabel("n_train")
plt.xlabel("Accuracy")
plt.legend()

# 3 - Bonus

* Rajouter un paramètre `var` à la fonction `generate_data` qui encode la covariance des données. On poura utiliser une matrice aléatoire $A$ et prendre $var = A^top A$ pour avoir une matrice psd. Visualiser les lignes de niveaux des fonctions de repartitions de des différentes classes.

* Implémenter l'aglorithme du plus proche centroid: au lieu de prendre la moyenne sur chaque classe, prendre le point dans l'ensemble de train tel que: $\bar X = \min_{x \in X_train} \sum_i \|x_i - x\|_2^2$.

* Implémenter l'algorithme des [k-plus proches voisins](https://fr.wikipedia.org/wiki/M%C3%A9thode_des_k_plus_proches_voisins). Dans ce cas, l'entrainement `fit` n'existe pas. Pour prédire le label de `x`, trouver les k points `idx_k` les plus proches de `x` dans `X_train` et retourner `y_hat` la classe majoritaire dans `y[idx_k]`. Comment les prédictions varient avec le paramètre `k`.